## Transfer learning
a. Create a new DNN that reuses all the pretrained hidden layers of the previous model, freezes them, and replaces the softmax output layer with a fresh new one.

b. Train this new DNN on digits 5 to 9, using only 100 images per digit, and time how long it takes. Despite this small number of examples, can you achieve high precision?

c. Try caching the frozen layers, and train the model again: how much faster is it now?

d. Try again reusing just four hidden layers instead of five. Can you achieve a higher precision?

e. Now unfreeze the top two hidden layers and continue training: can you get the model to perform even better?

In [1]:
import tensorflow as tf
import numpy as np

#### 9.1 Create a new DNN that reuses all the pretrained hidden layers of the previous model, freezes them, and replaces the softmax output layer with a fresh new one.

#### 9.2 Train this new DNN on digits 5 to 9, using only 100 images per digit, and time how long it takes. Despite this small number of examples, can you achieve high precision?

In [2]:
restore_saver = tf.train.import_meta_graph("/Users/chancezhang/machine_learning/hands_on_machine_learning/tmp/C11/C11_Ex08_100.ckpt.meta")
for op in tf.get_default_graph().get_operations():
        print(op.name)

X
y
is_training
hidden1/weights/Initializer/truncated_normal/shape
hidden1/weights/Initializer/truncated_normal/mean
hidden1/weights/Initializer/truncated_normal/stddev
hidden1/weights/Initializer/truncated_normal/TruncatedNormal
hidden1/weights/Initializer/truncated_normal/mul
hidden1/weights/Initializer/truncated_normal
hidden1/weights
hidden1/weights/Assign
hidden1/weights/read
hidden1/MatMul
hidden1/BatchNorm/Reshape/shape
hidden1/BatchNorm/Reshape
hidden1/BatchNorm/beta/Initializer/zeros
hidden1/BatchNorm/beta
hidden1/BatchNorm/beta/Assign
hidden1/BatchNorm/beta/read
hidden1/BatchNorm/Const
hidden1/BatchNorm/moving_mean/Initializer/zeros
hidden1/BatchNorm/moving_mean
hidden1/BatchNorm/moving_mean/Assign
hidden1/BatchNorm/moving_mean/read
hidden1/BatchNorm/moving_variance/Initializer/ones
hidden1/BatchNorm/moving_variance
hidden1/BatchNorm/moving_variance/Assign
hidden1/BatchNorm/moving_variance/read
hidden1/BatchNorm/cond/Switch
hidden1/BatchNorm/cond/switch_t
hidden1/BatchNorm/co

In [3]:
# Get original variables
# 由于一个Tensor可能会输出到很多地方，对于TF来说，每个输出输入节点都是唯一的，所以需要加编号?
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
is_training = tf.get_default_graph().get_tensor_by_name('is_training:0')
loss = tf.get_default_graph().get_tensor_by_name('loss/loss:0')
Y_probe = tf.get_default_graph().get_tensor_by_name('loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits:0')
logits = Y_probe.op.inputs[0]
accuracy = tf.get_default_graph().get_tensor_by_name('eval/accuracy:0')

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
# Create new training operation
output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='hidden5')
optimizer = tf.train.AdamOptimizer(name='Adam2')
train_op = optimizer.minimize(loss, var_list=output_layer_vars)

In [36]:
# Prepare small data set
X_train_all = mnist.train.images
y_train_all = mnist.train.labels
small_set_count = 100
selected_indices = []
for i in range(5, 10, 1):
    indices = np.where(y_train_all == i)    
    selected_indices.append(indices[0][:small_set_count])
X_train_small = X_train_all[np.array(selected_indices).flatten().tolist()]
# We need to subtract 5 from the labels because TensorFlow expects integers from 0 to n_classes-1.
y_train_small = y_train_all[np.array(selected_indices).flatten().tolist()] - 5

val_indices = np.where(mnist.test.labels > 4)
X_val = mnist.test.images[val_indices]
y_val = mnist.test.labels[val_indices] - 5

In [7]:
%%time
init = tf.global_variables_initializer()
model_path = '/Users/chancezhang/machine_learning/hands_on_machine_learning/tmp/C11/C11_Ex08_100.ckpt'
new_model_path ='/Users/chancezhang/machine_learning/hands_on_machine_learning/tmp/C11/C11_Ex08_200.ckpt'
reuse_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[12345]")
original_saver = tf.train.Saver(reuse_vars)

n_epoch = 2000

new_saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)    
    # 恢复原始模型的部分数据
    original_saver.restore(sess, model_path)
    for var in output_layer_vars:
        var.initializer.run()
    
    for epoch in range(n_epoch):
        sess.run(train_op, feed_dict={X:X_train_small, y:y_train_small, is_training:True})
        if epoch % 100 == 0:
            train_acc = accuracy.eval(feed_dict={X:X_train_small, y:y_train_small, is_training:False})
            val_acc = accuracy.eval(feed_dict={X:X_val, y:y_val, is_training:False})
            print('New Train Accuracy:', train_acc, 'Val Accuracy:', val_acc)
    
    new_saver.save(sess, new_model_path)

INFO:tensorflow:Restoring parameters from /Users/chancezhang/machine_learning/hands_on_machine_learning/tmp/C11/C11_Ex08_100.ckpt
New Train Accuracy: 0.162 Val Accuracy: 0.16560379
New Train Accuracy: 0.56 Val Accuracy: 0.5103888
New Train Accuracy: 0.754 Val Accuracy: 0.65624356
New Train Accuracy: 0.826 Val Accuracy: 0.71117055
New Train Accuracy: 0.848 Val Accuracy: 0.7362683
New Train Accuracy: 0.86 Val Accuracy: 0.7504629
New Train Accuracy: 0.874 Val Accuracy: 0.7566344
New Train Accuracy: 0.888 Val Accuracy: 0.762806
New Train Accuracy: 0.896 Val Accuracy: 0.76548034
New Train Accuracy: 0.898 Val Accuracy: 0.76650894
New Train Accuracy: 0.898 Val Accuracy: 0.7671261
New Train Accuracy: 0.894 Val Accuracy: 0.7650689
New Train Accuracy: 0.902 Val Accuracy: 0.76548034
New Train Accuracy: 0.904 Val Accuracy: 0.76424605
New Train Accuracy: 0.904 Val Accuracy: 0.7638346
New Train Accuracy: 0.906 Val Accuracy: 0.7646575
New Train Accuracy: 0.906 Val Accuracy: 0.7650689
New Train Accura

### 总结：
启用dropout之后，重新训练的准确度只能达到53%，弃用之后可以达到90%。因此如果需要重新训练的话，推荐不使用dropout

#### 9.3 Try caching the frozen layers, and train the model again: how much faster is it now?

In [23]:
%%time
# cache flozen layers
from tensorflow.contrib.layers import batch_norm

hidden4_output = tf.get_default_graph().get_tensor_by_name('hidden4/Relu:0')
with tf.Session() as sess:
    sess.run(init)    
    # 恢复原始模型的部分数据
    original_saver.restore(sess, model_path)
    for var in output_layer_vars:
        var.initializer.run()
    # is_training=False时精确度会下降？为什么，跟原模型有误差？
    hidden4_output_cache = hidden4_output.eval(feed_dict={X:X_train_small, is_training:True})
    hidden4_val = hidden4_output.eval(feed_dict={X:X_val, is_training:True})
    
    for epoch in range(n_epoch):
        sess.run(train_op, feed_dict={hidden4_output:hidden4_output_cache, y:y_train_small, is_training:True})
        if epoch % 100 == 0:
            # 为什么这里不能用X来作为输入进行验证？可以，但训练时必须也用X来训练。
            train_acc = accuracy.eval(feed_dict={hidden4_output:hidden4_output_cache, y:y_train_small, is_training:False})
            val_acc = accuracy.eval(feed_dict={hidden4_output:hidden4_val, y:y_val, is_training:False})
            print('New Train Accuracy:', train_acc, 'Val Accuracy:', val_acc)         

INFO:tensorflow:Restoring parameters from /Users/chancezhang/machine_learning/hands_on_machine_learning/tmp/C11/C11_Ex08_100.ckpt
New Train Accuracy: 0.13 Val Accuracy: 0.13783172
New Train Accuracy: 0.558 Val Accuracy: 0.5344579
New Train Accuracy: 0.746 Val Accuracy: 0.6650895
New Train Accuracy: 0.826 Val Accuracy: 0.70520467
New Train Accuracy: 0.852 Val Accuracy: 0.7350339
New Train Accuracy: 0.864 Val Accuracy: 0.7504629
New Train Accuracy: 0.892 Val Accuracy: 0.7640403
New Train Accuracy: 0.9 Val Accuracy: 0.76548034
New Train Accuracy: 0.902 Val Accuracy: 0.76630324
New Train Accuracy: 0.9 Val Accuracy: 0.76650894
New Train Accuracy: 0.904 Val Accuracy: 0.7669204
New Train Accuracy: 0.902 Val Accuracy: 0.7669204
New Train Accuracy: 0.902 Val Accuracy: 0.7671261
New Train Accuracy: 0.902 Val Accuracy: 0.7671261
New Train Accuracy: 0.902 Val Accuracy: 0.76630324
New Train Accuracy: 0.902 Val Accuracy: 0.7650689
New Train Accuracy: 0.9 Val Accuracy: 0.76548034
New Train Accuracy: 

#### 9.4 Now unfreeze the top two hidden layers and continue training: can you get the model to perform even better?

In [37]:
output_layer_vars_bottom3 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='hidden[345]')
optimizer2 = tf.train.AdamOptimizer(name='Adam3')
train_op2 = optimizer2.minimize(loss, var_list=output_layer_vars_bottom3)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    original_saver.restore(sess, model_path)
    for var in output_layer_vars_bottom3:
        var.initializer.run();
        
    for epoch in range(n_epoch):
        sess.run(train_op2, feed_dict={X:X_train_small, y:y_train_small, is_training:True})
        if epoch % 100 == 0:
            train_acc = accuracy.eval(feed_dict={X:X_train_small, y:y_train_small, is_training:False})
            val_acc = accuracy.eval(feed_dict={X:X_val, y:y_val, is_training:False})
            print('New Train Accuracy:', train_acc, 'Val Accuracy:', val_acc)
    

INFO:tensorflow:Restoring parameters from /Users/chancezhang/machine_learning/hands_on_machine_learning/tmp/C11/C11_Ex08_100.ckpt
New Train Accuracy: 0.13786069 Val Accuracy: 0.1419461
New Train Accuracy: 0.906906 Val Accuracy: 0.9022835
New Train Accuracy: 0.9398042 Val Accuracy: 0.92902696
New Train Accuracy: 0.95248866 Val Accuracy: 0.9389015
New Train Accuracy: 0.95964694 Val Accuracy: 0.9458959
New Train Accuracy: 0.9651732 Val Accuracy: 0.94959885
New Train Accuracy: 0.96958685 Val Accuracy: 0.95268464
New Train Accuracy: 0.9737779 Val Accuracy: 0.9551533
New Train Accuracy: 0.9776723 Val Accuracy: 0.9576219
New Train Accuracy: 0.98045397 Val Accuracy: 0.9572104
New Train Accuracy: 0.9831615 Val Accuracy: 0.9572104
New Train Accuracy: 0.9854239 Val Accuracy: 0.9594734
New Train Accuracy: 0.9873897 Val Accuracy: 0.96050197
New Train Accuracy: 0.98887324 Val Accuracy: 0.96050197
New Train Accuracy: 0.99050516 Val Accuracy: 0.96050197
New Train Accuracy: 0.9915066 Val Accuracy: 0.96

### 总结：
1. 冻结了两层，重新训练3层之后，准确度有所提高！
2. 50x5个用例训练精度可以提高到82%， 20x5个用例训练精度可以提高到74%，说明重新训练是可以很好地节省时长的